In [46]:
import os
import sys
import pandas as pd
from datetime import date, timedelta
from pymongo import MongoClient

caminho_commons = os.path.join('..', '..', 'ajna_docs', 'commons')
caminho_virasana = os.path.join('..', '..', 'virasana')
caminho_bhadrasana = os.path.join('..', '..', 'bhadrasana')
sys.path.append(caminho_commons)
sys.path.append('..')
sys.path.append(caminho_bhadrasana)
sys.path.append(caminho_virasana)

In [47]:
from bhadrasana.models import engine
from bhadrasana.models.virasana_manager import get_imagens_container
from virasana.integracao.mercante.mercantealchemy import Item
from ajna_commons.flask.conf import MONGODB_URI, DATABASE

conn = MongoClient(host=MONGODB_URI)
mongodb = conn[DATABASE]

In [48]:
df_paises = pd.read_csv('data/Paises.csv', sep=';')
df_paises

,sigla,nome,escaneamento
0,AE,Emirados Árabes Unidos,1.0
1,AG,Antígua e Barbuda,NaN
2,AL,Albânia,1.0
3,AN,Antilhas Holandesas,NaN
4,AO,Angola,1.0
...,...,...,...
143,VE,Venezuela,NaN
144,VI,Ilhas Virgens (U.S.),NaN
145,VN,Vietnã,NaN
146,YE,Iêmen,NaN


In [49]:
df_paises['escaneamento'] = df_paises['escaneamento'].apply(lambda x: x == 1.0)
df_paises

,sigla,nome,escaneamento
0,AE,Emirados Árabes Unidos,True
1,AG,Antígua e Barbuda,False
2,AL,Albânia,True
3,AN,Antilhas Holandesas,False
4,AO,Angola,True
...,...,...,...
143,VE,Venezuela,False
144,VI,Ilhas Virgens (U.S.),False
145,VN,Vietnã,False
146,YE,Iêmen,False


In [50]:
#df_paises.to_sql('risco_paises', engine, index=False)  #, if_exists='append')

In [51]:
df_paises = pd.read_sql('SELECT * FROM risco_paises', engine)

In [52]:
df_paises_escaneados = df_paises[df_paises.escaneamento == True]
df_paises_escaneados

,sigla,nome,escaneamento
0,AE,Emirados Árabes Unidos,1
2,AL,Albânia,1
4,AO,Angola,1
10,BE,Bélgica,1
11,BG,Bulgária,1
...,...,...,...
133,TN,Tunísia,1
134,TR,Turquia,1
137,TZ,República Unida da Tanzânia,1
138,UA,Ucrânia,1


In [53]:
df_paises_escaneados[df_paises_escaneados.sigla == 'HK']

,sigla,nome,escaneamento


In [54]:
SQL_ESCANEAMENTOS = \
'''
SELECT c.numeroCEmercante, substring(c.portoDestFinal, 1, 2) as pais_porto_final, 
substring(m.portoDescarregamento, 1, 2) as pais_porto_baldeacao 
FROM conhecimentosresumo c INNER JOIN manifestosresumo m ON c.manifestoCE = m.numero
WHERE c.dataEmissao between '{}' and '{}'
AND c.portoOrigemCarga = '{}'
'''
SQL_OBRIGATORIOS = SQL_ESCANEAMENTOS + \
'''
AND (
(substring(c.portoDestFinal, 1, 2) IN (SELECT sigla FROM risco_paises WHERE escaneamento is True))
OR 
(substring(m.portoDescarregamento, 1, 2) IN (SELECT sigla FROM risco_paises WHERE escaneamento is True))
)'''

In [55]:
from datetime import datetime, timedelta

inicio = datetime(2023, 7, 1) 
fim = datetime(2023, 7, 2) 
inicio_scan = inicio - timedelta(days=10)
fim_scan = fim + timedelta(days=10)

print(SQL_ESCANEAMENTOS.format(inicio, fim, 'BRSSZ'))
df_embarques = pd.read_sql(SQL_ESCANEAMENTOS.format(inicio, fim, 'BRSSZ'), engine)
df_embarques


SELECT c.numeroCEmercante, substring(c.portoDestFinal, 1, 2) as pais_porto_final, 
substring(m.portoDescarregamento, 1, 2) as pais_porto_baldeacao 
FROM conhecimentosresumo c INNER JOIN manifestosresumo m ON c.manifestoCE = m.numero
WHERE c.dataEmissao between '2023-07-01 00:00:00' and '2023-07-02 00:00:00'
AND c.portoOrigemCarga = 'BRSSZ'



,numeroCEmercante,pais_porto_final,pais_porto_baldeacao
0,152307141109966,PH,PH
1,152307148376919,KR,KR
2,152307148377133,KR,KR
3,152307159031403,US,US
4,152307159032566,US,US
...,...,...,...
1075,152307172776700,US,US
1076,152307172776972,US,US
1077,152307172781380,US,US
1078,152307174159920,US,US


In [56]:
df_embarques_a_escanear = df_embarques[df_embarques.pais_porto_final.isin(df_paises_escaneados.sigla.values)]
df_embarques_a_escanear

,numeroCEmercante,pais_porto_final,pais_porto_baldeacao
703,152307161169913,AE,SG
712,152307161700236,BE,CO
723,152307161701470,BE,CO
739,152307161703413,BE,MX
740,152307161703502,DE,MX
764,152307163190863,AE,AE
765,152307163191592,AE,AE
805,152307164394136,CI,CO
806,152307164394217,CI,CO
808,152307164394489,GH,CO


In [57]:
df_embarques_a_escanear = df_embarques[df_embarques.pais_porto_baldeacao.isin(df_paises_escaneados.sigla.values)]
df_embarques_a_escanear

,numeroCEmercante,pais_porto_final,pais_porto_baldeacao
764,152307163190863,AE,AE
765,152307163191592,AE,AE


In [58]:
df_embarques_a_escanear = df_embarques[df_embarques.pais_porto_final.isin(df_paises_escaneados.sigla.values) |
             df_embarques.pais_porto_baldeacao.isin(df_paises_escaneados.sigla.values)]
df_embarques_a_escanear

,numeroCEmercante,pais_porto_final,pais_porto_baldeacao
703,152307161169913,AE,SG
712,152307161700236,BE,CO
723,152307161701470,BE,CO
739,152307161703413,BE,MX
740,152307161703502,DE,MX
764,152307163190863,AE,AE
765,152307163191592,AE,AE
805,152307164394136,CI,CO
806,152307164394217,CI,CO
808,152307164394489,GH,CO


In [59]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

In [60]:
query = {'metadata.numeroinformado': 'BEAU5395087',
         'metadata.contentType': 'image/jpeg',
         'metadata.dataescaneamento': {'$gte': inicio_scan, '$lte': fim_scan}
         }
projection = {'metadata.numeroinformado': 1,
              'metadata.dataescaneamento': 1,
             'metadata.predictions.vazio': 1}
cursor = mongodb['fs.files'].find(query, projection)
list(cursor)

[{'_id': ObjectId('64944b4a51f557bdb592ef66'),
  'metadata': {'dataescaneamento': datetime.datetime(2023, 6, 21, 6, 48, 39),
   'numeroinformado': 'BEAU5395087',
   'predictions': [{'vazio': True}]}},
 {'_id': ObjectId('64a21c468086181d70d2c061'),
  'metadata': {'dataescaneamento': datetime.datetime(2023, 7, 1, 5, 10, 35),
   'numeroinformado': 'BEAU5395087',
   'predictions': [{'vazio': False}]}},
 {'_id': ObjectId('64a815d37198925d2b1edb6c'),
  'metadata': {'dataescaneamento': datetime.datetime(2023, 7, 6, 19, 51, 39),
   'numeroinformado': 'BEAU5395087',
   'predictions': [{'vazio': False}]}}]

In [61]:
def get_imagens_container_data(mongodb, numero, inicio_scan, fim_scan, vazio=False) -> list:
    query = {'metadata.numeroinformado': numero,
             'metadata.contentType': 'image/jpeg',
             'metadata.dataescaneamento': {'$gte': inicio_scan, '$lte': fim_scan}
             }
    projection = {'metadata.numeroinformado': 1,
                  'metadata.dataescaneamento': 1,
                 'metadata.predictions.vazio': 1}
    cursor = mongodb['fs.files'].find(query, projection)
    return list(cursor)

In [62]:
vazios = 0
semimagem = 0
total = 0
print(f'Checando embarques (emissão CE) entre {inicio} e {fim} com escaneamentos entre {inicio_scan} e {fim_scan}')

for cemercante in df_embarques_a_escanear.numeroCEmercante.values:
    # print(cemercante)
    itens = session.query(Item).filter(Item.numeroCEmercante == cemercante).all()
    for item in itens:
        total += 1
        # print(item.codigoConteiner)
        imagens = get_imagens_container_data(mongodb, item.codigoConteiner, inicio_scan, fim_scan)
        if len(imagens) == 0:
            semimagem += 1
            print(f'{item.codigoConteiner} CE {cemercante} sem imagem com escaneamento entre {inicio_scan} e {fim_scan}!!')
        for imagem in imagens:
            if imagem['metadata']['predictions'][0]['vazio'] is False:
                vazio = False
                break
            vazio = True
        if vazio:
            vazios += 1
            print(f'{item.codigoConteiner} com imagem supostamente de vazio!')

print(f'De {total} contêineres, de {len(df_embarques_a_escanear)} CEs, '
      f'{semimagem} sem imagem e {vazios} com suposta imagem de vazio.')

Checando embarques (emissão CE) entre 2023-07-01 00:00:00 e 2023-07-02 00:00:00 com escaneamentos entre 2023-06-21 00:00:00 e 2023-07-12 00:00:00
FCIU5543607 CE 152307161169913 sem imagem com escaneamento entre 2023-06-21 00:00:00 e 2023-07-12 00:00:00!!
NYKU9748332 CE 152307161701470 sem imagem com escaneamento entre 2023-06-21 00:00:00 e 2023-07-12 00:00:00!!
TCLU7311785 CE 152307161701470 sem imagem com escaneamento entre 2023-06-21 00:00:00 e 2023-07-12 00:00:00!!
De 52 contêineres, de 14 CEs, 3 sem imagem e 0 com suposta imagem de vazio.
